In [1]:
import time

while True:

    from datetime import datetime
    from influxdb_client import InfluxDBClient, Point, WritePrecision
    from influxdb_client.client.write_api import SYNCHRONOUS
    import pandas as pd
    from datetime import datetime, timedelta
    import numpy as np
    from fbprophet import Prophet
    from pandas import to_datetime
    from pandas import DataFrame


    t = "C3GlPOdiT0dqAH9LH_Q8Z0vtXdkZ84DAkFE0phwEtgR_lbc7TYP4Sz6rTavhbNcLIjT9joY15Lyz4x1kkd30LA=="
    o = "project"
    b = "binance"
    m = "mess"
    datetime_format = '%Y-%m-%d %H:%M:%S'
    prediction_time_in_secs = 5 * 60
    sleep_time_in_secs = 4 * 60

    predicted_m = "mess_p"
    client = InfluxDBClient(url="http://localhost:8086", token=t, org=o)

    query_api = client.query_api()
    write_api = client.write_api(write_options=SYNCHRONOUS)

    def read_from_influx(coin):
        tables = query_api.query(f'from(bucket: "{b}") |> range(start: -1h,stop: now()) |> filter(fn: (r) => r._measurement == "{m}") |> filter(fn: (r) => r.coin == "{coin}")', org=o)
        resultset = []
        for table in tables:
            for row in table.records:
                resultset.append({'time':row["_time"].strftime(datetime_format),'value':row["_value"],'coin':row["coin"]})
        df = pd.DataFrame(resultset, columns=["time", "value", "coin"])
        return df


    def write_in_influx(df, column_name):
        for ind in df.index:
            p = Point(predicted_m).tag("coin", df['coin'][ind]).field(column_name, df[column_name][ind]).time(int(df['time'][ind]/1000000), WritePrecision.MS)
            write_api.write(bucket=b, record=p)


    coin = "BTCBUSD"
    df = read_from_influx(coin)
    
    df_train = df[['time','value']]
    df_train['ds'] = df_train['time']
    df_train['y'] = df_train['value']
    df_train.drop(['time', 'value'], axis='columns', inplace=True)
    print(df_train)

    model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
    model.fit(df_train)


    last_t = df_train['ds'].iloc[-1]

    future = list()
    for i in range(1, prediction_time_in_secs):
        x = to_datetime(last_t) + timedelta(seconds=i)
        future.append(x)

    future = DataFrame(future)
    future.columns = ['ds']
    future['ds'] = to_datetime(future['ds'])

    print(future['ds'])

    forecast = model.predict(future)
    future['time'] = future['ds'].values.astype(np.int64)
    future['yhat'] = forecast['yhat']
    future['yhat_lower'] = forecast['yhat_lower']
    future['yhat_upper'] = forecast['yhat_upper']
    future['coin'] = coin

    print(future)

    write_in_influx(future, 'yhat')
    print('yhat completed')
    write_in_influx(future, 'yhat_lower')
    print('yhat_lower completed')
    write_in_influx(future, 'yhat_upper')
    print('yhat_upper completed')
    client.close()
    
    time.sleep(sleep_time_in_secs)



                       ds         y
0     2022-03-29 06:04:05  47048.41
1     2022-03-29 06:04:06  47048.41
2     2022-03-29 06:04:07  47048.42
3     2022-03-29 06:04:08  47048.42
4     2022-03-29 06:04:09  47048.41
...                   ...       ...
1751  2022-03-29 06:54:28  46898.00
1752  2022-03-29 06:54:29  46897.99
1753  2022-03-29 06:54:30  46898.00
1754  2022-03-29 06:54:31  46897.99
1755  2022-03-29 06:54:32  46897.99

[1756 rows x 2 columns]


/opt/anaconda3/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -2.02618
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99         12144    0.00786941        901354           1           1      129   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       12479.8    0.00241206       36705.3     0.08296           1      247   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       12531.3   0.000449491       58914.1           1           1      369   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       12672.7     0.0220096        482189       2.105     0.02105      486   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       12784.6     0.0017549        766305      0.2092      0.2092      617   
    Iter      log prob        ||dx||      ||grad||       alpha  

/opt/anaconda3/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/opt/anaconda3/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


                     ds                 time          yhat    yhat_lower  \
0   2022-03-29 06:54:33  1648536873000000000  46868.896818  46846.864115   
1   2022-03-29 06:54:34  1648536874000000000  46868.673850  46848.687894   
2   2022-03-29 06:54:35  1648536875000000000  46868.451082  46847.848198   
3   2022-03-29 06:54:36  1648536876000000000  46868.228515  46848.229947   
4   2022-03-29 06:54:37  1648536877000000000  46868.006148  46846.315058   
..                  ...                  ...           ...           ...   
294 2022-03-29 06:59:27  1648537167000000000  46812.001858  46588.977991   
295 2022-03-29 06:59:28  1648537168000000000  46811.838021  46577.587637   
296 2022-03-29 06:59:29  1648537169000000000  46811.674385  46587.765659   
297 2022-03-29 06:59:30  1648537170000000000  46811.510951  46583.966786   
298 2022-03-29 06:59:31  1648537171000000000  46811.347718  46578.280826   

       yhat_upper     coin  
0    46890.985449  BTCBUSD  
1    46889.685377  BTCBUSD  


...


Initial log joint probability = -96.0273
Iteration  1. Log joint probability =    208.579. Improved by 304.607.
Iteration  2. Log joint probability =    237.359. Improved by 28.7793.
Iteration  3. Log joint probability =    256.209. Improved by 18.8506.
Iteration  4. Log joint probability =    283.056. Improved by 26.8463.
Iteration  5. Log joint probability =    293.452. Improved by 10.3969.
Iteration  6. Log joint probability =    331.118. Improved by 37.6659.
Iteration  7. Log joint probability =    355.925. Improved by 24.807.
Iteration  8. Log joint probability =    381.629. Improved by 25.7037.
Iteration  9. Log joint probability =     405.68. Improved by 24.0508.
Iteration 10. Log joint probability =    412.979. Improved by 7.29887.
Iteration 11. Log joint probability =    419.144. Improved by 6.16486.
Iteration 12. Log joint probability =    419.756. Improved by 0.612053.
Iteration 13. Log joint probability =     420.15. Improved by 0.394658.
Iteration 14. Log joint probabilit

/opt/anaconda3/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/opt/anaconda3/lib/python3.8/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


                     ds                 time          yhat    yhat_lower  \
0   2022-03-29 07:24:24  1648538664000000000  46989.036314  46987.725172   
1   2022-03-29 07:24:25  1648538665000000000  46988.170576  46984.067689   
2   2022-03-29 07:24:26  1648538666000000000  46987.232491  46979.180480   
3   2022-03-29 07:24:27  1648538667000000000  46986.222059  46973.310648   
4   2022-03-29 07:24:28  1648538668000000000  46985.139280  46966.996606   
..                  ...                  ...           ...           ...   
294 2022-03-29 07:29:18  1648538958000000000  43620.091347  34015.505084   
295 2022-03-29 07:29:19  1648538959000000000  43597.978174  33922.125682   
296 2022-03-29 07:29:20  1648538960000000000  43575.792883  33819.040849   
297 2022-03-29 07:29:21  1648538961000000000  43553.535484  33718.535966   
298 2022-03-29 07:29:22  1648538962000000000  43531.205973  33644.311106   

       yhat_upper     coin  
0    46990.531849  BTCBUSD  
1    46991.969412  BTCBUSD  


/var/folders/np/7wstqrjj7h77xmr2lv_bt64h0000gn/T/ipykernel_11963/1921404838.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(['time', 'value'], axis='columns', inplace=True)


ValueError: Dataframe has less than 2 non-NaN rows.